In [0]:
# Databricks Notebook: 2_Sales_Analysis
# This notebook performs KPIs calculations, aggregation over time, and identifies sales trends.

from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, avg, count, desc

spark = SparkSession.builder.appName("EcommerceSalesAnalysis").getOrCreate()

# Read the cleaned data from the Delta table
df = spark.sql("SELECT * FROM ecommerce_sales_cleaned")
display(df)

order_id,sale_date,price,product_id,category,year
O1001,2023-01-15,120.5,P101,Electronics,2023
O1001,2023-01-15,45.0,P102,Books,2023
O1001,2023-01-15,85.99,P103,Clothing,2023
O1002,2023-01-16,50.0,P102,Books,2023
O1002,2023-01-16,110.0,P104,Home,2023
O1002,2023-01-16,99.99,P105,Beauty,2023
O1002,2023-01-16,210.0,P101,Electronics,2023
O1003,2023-01-17,75.0,P103,Clothing,2023
O1003,2023-01-17,60.0,P106,Sports,2023
O1004,2023-01-18,120.0,P101,Electronics,2023


In [0]:
# Compute total revenue per year
revenue_per_year = df.groupBy("year").agg(sum("price").alias("total_revenue"))
display(revenue_per_year)

year,total_revenue
2023,4895.899982452393


In [0]:
# Calculate average order value and total orders per product category (assuming there's a category column)
if "category" in df.columns:
    category_stats = df.groupBy("category")\
                       .agg(count("product_id").alias("total_orders"), avg("price").alias("avg_order_value"))\
                       .orderBy(desc("total_orders"))
    display(category_stats)

category,total_orders,avg_order_value
Clothing,11,67.97272664850408
Home,10,164.575
Electronics,7,134.57142857142858
Books,6,44.25
Sports,5,72.0
Garden,5,65.0
Beauty,5,99.98999786376953
Toys,5,22.0


In [0]:
# Further analysis: Top 10 products by revenue
top_products = df.groupBy("product_id")\
                 .agg(sum("price").alias("product_revenue"))\
                 .orderBy(desc("product_revenue"))\
                 .limit(10)
display(top_products)

product_id,product_revenue
P109,1250.0
P101,942.0
P105,499.94998931884766
P103,468.9499931335449
P104,395.75
P106,360.0
P110,325.0
P108,278.75
P102,265.5
P107,110.0
